# Fourier Transformation


## Introduction

In this notebook we will learn how to apply the 1D Fourier Transformation (FT). The Fourier Transformation is often used to do the transformation from the time (or spatial) domain to the frequency domain (and backwards). The goal of the FT is to investigate the frequency content of the input data. The frequencies and their amplitudes are combined to create the fourier amplitude spectrum.


This process is done in the easiest way by the fourier series (FS, [Wiki](https://en.wikipedia.org/wiki/Fourier_series) or [simpleWiki](https://simple.wikipedia.org/wiki/Fourier_transform)). Hereby, (multiple) sines and cosines with different frequencies are sumed, trying to recreate the original signals. Considering the things learned in the previous notebook about models, we can also think of the Fourier Transform as such a model. In the Fourier Series the base functions are the trigonometric functions:

$y = a \cdot sin(b\cdot x + \phi_b) + c \cdot sin(d\cdot x + \phi_d) + ... + e \cdot cos(f\cdot x + \phi_f) + ...$

$y = \sum_{i=0}^{N} a_i \cdot sin(w_i\cdot x + \phi_i) + b_i \cdot cos(w_i\cdot x + \phi_i)$

For the FT this sum is replaced by the integral. The trigonometric functions are displayed as Euler's formula.
The final form of the Fourier Transform:

$F(\tau) = \int_{-\inf}^{\inf} f(t) \cdot e^{-2\pi i \tau t} \cdot dt$ 

In depth explanation can be found, e.g. [here](https://www.thefouriertransform.com/). And there is a ton of videos explaining the Fourier Transform in different ways, e.g. [3Blue1Brown](https://www.youtube.com/watch?v=spUNpyF58BY)

In computational science the FS/FT is realized by the discrete fourier transform (DFT), often used in combination with the fast fourier transform (FFT) algorithm. The FFT allows for the fast and efficient calculation.


The FT is used/needed for the following lectures:
- Digital Seismology
- almost everywhere



### Table of Contents
- [Fast Fourier Transform](#Fast_Fourier_Transform)
- [Testing Different Frequencies](#Testing_Different_Frequencies)
- [Multiple Frequencies](#Multiple_Frequencies)
- [Specials](#Specials)
- [Full Fourier Transform](#Full)
- [Summary](#Summary)


<a id='Fast_Fourier_Transform'></a> 
# Fast Fourier Transform

In [ ]:
import numpy as num
import scipy as sp
import matplotlib.pyplot as plt

First we create some synthetic time signal. For that we need to define its sampling frequency (df), meaning how many data points we want to have within one second. We also require the inverse of the sampling frequency, the sampling intveral (dt), indicating the time between two samples, and the length of our time trace. Important to note: the sampling must be equidistant between samples.

For the beginning we choose as function a sinus with a frequency of `fr1 = 1 Hz`, meaning one full oscillation every second.

In [ ]:
df = 20  # Sampling frequency [Hz]
timelen = 10  # Data length [s]
dt = 1 / df  # Sampling interval [s]

numsamples = timelen * df
x = num.arange(numsamples) / df
#print(x)
fr1 = 1 # frequency of signal  [Hz], inverse of periode
ydata = num.sin(2 * num.pi * fr1 * x)
plt.figure(figsize=(12, 3))
plt.plot(x, ydata)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.grid()
plt.show()

The FT produces a amplitude spectrum and the phase spectrum. In most of the cases the phase information is not relevant, as in general, its interpretation is hard. Therefore, it is often neglected (not shown). When someone mentions fourier spectrum usually they refer to the amplitude spectrum. 

Next, we apply the FT in Numpy. We can use either the ['fft'](https://numpy.org/doc/stable/reference/generated/numpy.fft.fft.html) (full spectrum, as complex values) or  ['rfft'](https://numpy.org/doc/stable/reference/generated/numpy.fft.rfft.html) (real values only). For most cases the `rfft` is sufficient as the spectrum is often symmetric (see further down). 

We need the correct information about the frequencies. Therefore, we can either calculate the frequencies manually by taking the sampling frequency and the signal length into account or by giving this information to a helper function of `numpy`: `rfftfreq`. It returns the correct frequencies to our previously gathered amplitudes.


In [ ]:
# Apply the ft - fft stands for fast-fourier-transform
ft = num.fft.rfft(ydata)

# To get the amplitude spectrum, only the real values are of interest - absolute
amps = abs(ft) * dt
phase =  num.angle(ft)  / num.pi

# Creating the corresponding frequencies
freqs = num.fft.rfftfreq(numsamples, d=dt)

plt.figure(figsize=(12, 6))
ax = plt.subplot(2, 1, 1)
ax.stem(freqs, amps)
# ax.set_xscale('log')
# ax.set_yscale('log')
# ax.plot(freqs, amps)
#ax.semilogx(freqs, amps, 'b.:')
#ax.loglog(freqs, amps, 'b.:')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Amplitude')
ax.set_title('Amplitude-spectrum')

ax = plt.subplot(2, 1, 2)
ax.stem(freqs, phase)
ax.phase_spectrum(ydata, marker='.', Fs=1/dt, color='orange')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Phase')
ax.set_title('Phase-spectrum')
plt.tight_layout()
plt.show()

Usually, the amplitudes of the spectrum are interpreted in relation to the rest of the spectrum. Therefore the absolute amplitudes are often not of importance.

There is also the inverse-transformation: from frequency domain to the time domain, e.g. [irfft](https://numpy.org/doc/stable/reference/generated/numpy.fft.irfft.html#numpy.fft.irfft).

<a id='Testing_Different_Frequencies'></a> 
# Testing Different frequencies

<div class ="alert alert-success">
Tasks
    
- play with the frequency of the sinus

- play with different sampling frequencies

- change the sinus and cosine

- add noise to the data
</div>

    
<div class ="alert alert-warning">
Take into account that the valid frequencies depend on your data:

- The highest possible frequency that can be displayed depends on the sampling rate. Here the [Nyquist(-Shannon)-Theorem](https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem) plays an important role. In short: frequencies higher than half of your sampling rate cannot be displayed correctly. That is why your x-axis in the example only shows maximum of df/2. Thus, the higher our sampling rate the higher the frequencies we can investigate.
    - Try frequencies: e.g. $\pm$ 10 Hz of your Nyquist-Frequency

    
- The lowest possible frequency depends on the length of your data.
     - Try frequencies: lower than 1/(maximum seconds of your data) Hz
</div>

In [ ]:
df = 20  # Sampling frequency [Hz]
timelen = 10  # Data length [s]
dt = 1 / df  # Sampling interval [s]

numsamples = timelen * df
x = num.arange(numsamples) / df
#print(x)

fr1 = 1 # frequency of signal  [Hz]
ydata = num.sin(2 * num.pi * fr1 * x) # + num.random.normal(0, 0.1, len(x))
# ydata = num.cos(2 * num.pi * fr1 * x) # + num.random.normal(0, 0.1, len(x))

# Get amplitudes and frequencies
ft = num.fft.rfft(ydata)
amps = abs(ft) * dt
phase =  num.angle(ft)  / num.pi
freqs = num.fft.rfftfreq(numsamples, d=dt)

plt.figure(figsize=(12, 9))
ax = plt.subplot(3, 1, 1)
ax.plot(x, ydata)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Amplitude')
ax.set_title('Time-domain')

ax = plt.subplot(3, 1, 2)
ax.stem(freqs, amps)
# ax.set_xscale('log')
# ax.set_yscale('log')
# ax.plot(freqs, amps)
#ax.semilogx(freqs, amps, 'b.:')
#ax.loglog(freqs, amps, 'b.:')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Amplitude')
ax.set_title('Frequency-domain')
plt.tight_layout()

ax = plt.subplot(3, 1, 3)
ax.stem(freqs, phase)
# ax.phase_spectrum(ydata, marker='.', Fs=1/dt, color='orange')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Phase')
ax.set_title('Phase-spectrum')
plt.tight_layout()
plt.show()

The amplitude spectrum of the FT does not distinguish between cosine and sinus, as it only evaluates the frequency content of a signal, independent of its origin. That means that two different waveforms can produce the same amplitude spectrum. As the FFT and its inverse are "unique and reversable" functions, the differences between sinus and cosine are only present in their phase spectra. 


<a id='Multiple_Frequencies'></a> 
# Multiple Frequencies
To investigate what the Fourier Transformation is really capable of, we add multiple sinus with different frequencies. The signal and its frequency content (usually) cannot be visually decomposed by us directly. Here the FFT can help us.

In [ ]:
df = 50  # Sampling frequency [Hz]
timelen = 10  # Data length [s]
dt = 1 / df  # Sampling interval [s]

numsamples = timelen * df
x = num.arange(numsamples) / df

fr1 = 1.
fr2 = 10.
ydata = num.sin(2 * num.pi * fr1 * x) 
ydata += 2*num.sin(2 * num.pi * fr2 * x)

# Get amplitudes and frequencies
ft = num.fft.rfft(ydata)
amps = abs(ft) * dt
phase =  num.angle(ft)  / num.pi
freqs = num.fft.rfftfreq(numsamples, d=dt)

plt.figure(figsize=(12, 9))
ax = plt.subplot(3, 1, 1)
ax.plot(x, ydata)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Amplitude')
ax.set_title('Time-domain')

ax = plt.subplot(3, 1, 2)
ax.stem(freqs, amps)
# ax.set_xscale('log')
# ax.set_yscale('log')
# ax.plot(freqs, amps)
#ax.semilogx(freqs, amps, 'b.:')
#ax.loglog(freqs, amps, 'b.:')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Amplitude')
ax.set_title('Frequency-domain')
plt.tight_layout()


ax = plt.subplot(3, 1, 3)
ax.stem(freqs, phase)
# ax.phase_spectrum(ydata, marker='.', Fs=1/dt, color='orange')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Phase')
ax.set_title('Phase-spectrum')
plt.tight_layout()

plt.show()

<a id='Specials'></a> 
# Specials
In the following some special cases are shown to demonstrate what the FT is capable of.

E.g. 
- a peak in time corresponds to a horizontal line in frequency (all frequencies)
- a lot of frequencies

In [ ]:
df = 100 # Sampling frequency [Hz]
timelen = 10 # Data length [s]
dt = 1 / df  # Sampling interval [s]

numsamples = timelen * df
x = num.arange(numsamples) / df

# Creating a array full of zeros
ydata = num.zeros(numsamples)

## single peak
ydata[0] = 1

## box
# ydata[450:550] = 1#

## multi special
# ydata = num.zeros(numsamples)
# frequencies = num.linspace(1, df/2, int(df/2))
# for ff in frequencies:
#     ydata += (1/ff) * num.sin(2 * num.pi * ff * x)
# #     ydata += ff * num.sin(2 * num.pi * ff * x)


ft = num.fft.rfft(ydata)
amps = abs(ft) * dt
phase =  num.angle(ft)  / num.pi
freqs = num.fft.rfftfreq(numsamples, d=dt)

plt.figure(figsize=(12, 9))
ax = plt.subplot(3, 1, 1)
ax.plot(x, ydata)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Amplitude')
ax.set_title('Time-domain')

ax = plt.subplot(3, 1, 2)
# ax.stem(freqs, amps)
# ax.set_xscale('log')
# ax.set_yscale('log')
ax.stem(freqs, amps)
#ax.semilogx(freqs, amps, 'b.:')
#ax.loglog(freqs, amps, 'b.:')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Amplitude')
ax.set_title('Frequency-domain')
plt.tight_layout()


ax = plt.subplot(3, 1, 3)
ax.stem(freqs, phase)
# ax.phase_spectrum(ydata, marker='.', Fs=1/dt, color='orange')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Phase')
ax.set_title('Phase-spectrum')
plt.tight_layout()

plt.show()

<a id='Full'></a> 
# Full Fourier Transform
As a last examle we look at the full Fourier Transform, which is actually a complex signal and not only the real values. The reason why we often neglect the complex form is that the spectrum is symmetric. But there are signals that produce non-symmetric spectrums, and therefore only the full FT is capable resolving those signals.

In [ ]:
df = 25 # Sampling frequency [Hz]
timelen = 5 # Data length [s]
dt = 1 / df  # Sampling interval [s]

numsamples = timelen * df
x = num.arange(numsamples) / df

fr1 = 1.
fr2 = 2.
ydata = num.sin(2 * num.pi * fr1 * x)
ydata += 2*num.sin(2 * num.pi * fr2 * x)

ft = num.fft.fft(ydata)
amps = abs(ft) * dt
phase =  num.angle(ft)  / num.pi
freqs = num.fft.fftfreq(numsamples, d=dt)

plt.figure(figsize=(12, 9))
ax = plt.subplot(3, 1, 1)
ax.plot(x, ydata)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Amplitude')
ax.set_title('Time-domain')

ax = plt.subplot(3, 1, 2)
ax.stem(freqs, amps)
# ax.set_xscale('log')
# ax.set_yscale('log')
# ax.plot(freqs, amps)
#ax.semilogx(freqs, amps, 'b.:')
#ax.loglog(freqs, amps, 'b.:')
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Amplitude [..]')
ax.set_title('Frequency-domain')
plt.tight_layout()


ax = plt.subplot(3, 1, 3)
ax.stem(freqs, phase)
ax.set_xlabel('Frequency [Hz]')
ax.set_ylabel('Phase')
ax.set_title('Phase-spectrum')
plt.tight_layout()

plt.show()

<a id='Summary'></a> 
# Summary

We have learned 
- the basic behaviour of the fourier transformation that it analyzes the frequency content of a signal
- different signals and their corresponding spectrum